# Module 8: Image Generation and GANs

- Generative Adversarial Networks (GANs)
- - Generator vs Discriminator.
- Conditional GANs (cGANs)
- - Generate gambar berdasarkan label.
- Image-to-Image Translation
- - Contoh: Pix2Pix (foto → sketsa).
- DeepDream and Artistic Style Transfer
- - Mengubah foto menjadi gaya lukisan.

In [ ]:
!pip install tensorflow matplotlib --quiet
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

# Load dataset MNIST
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1).astype("float32")
BUFFER_SIZE = 60000
BATCH_SIZE = 256

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

num_classes = 10
noise_dim = 100

# Generator dengan label
def build_generator():
    noise = layers.Input(shape=(noise_dim,))
    label = layers.Input(shape=(1,), dtype="int32")

    label_embedding = layers.Embedding(num_classes, noise_dim)(label)
    label_embedding = layers.Flatten()(label_embedding)

    model_input = layers.multiply([noise, label_embedding])

    x = layers.Dense(7*7*256, use_bias=False)(model_input)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((7,7,256))(x)
    x = layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    img = layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding="same", use_bias=False, activation="tanh")(x)

    return tf.keras.Model([noise, label], img)

# Discriminator dengan label
def build_discriminator():
    img = layers.Input(shape=(28,28,1))
    label = layers.Input(shape=(1,), dtype="int32")

    label_embedding = layers.Embedding(num_classes, 28*28)(label)
    label_embedding = layers.Flatten()(label_embedding)
    label_embedding = layers.Reshape((28,28,1))(label_embedding)

    x = layers.Concatenate(axis=-1)([img, label_embedding])
    x = layers.Conv2D(64, (5,5), strides=(2,2), padding="same")(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128, (5,5), strides=(2,2), padding="same")(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    validity = layers.Dense(1)(x)

    return tf.keras.Model([img, label], validity)

generator = build_generator()
discriminator = build_discriminator()

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
disc_optimizer = tf.keras.optimizers.Adam(1e-4)

# Loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Training step
@tf.function
def train_step(images, labels):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator([noise, labels], training=True)
        real_output = discriminator([images, labels], training=True)
        fake_output = discriminator([generated_images, labels], training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

# Fungsi generate gambar
def generate_and_plot(generator, epoch, label):
    noise = tf.random.normal([16, noise_dim])
    labels = tf.constant([label]*16)
    predictions = generator([noise, labels], training=False)

    plt.figure(figsize=(4,4))
    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap="gray")
        plt.axis("off")
    plt.suptitle(f"Epoch {epoch} - Label {label}")
    plt.show()

# Training loop singkat
EPOCHS = 3
for epoch in range(EPOCHS):
    for image_batch, label_batch in dataset:
        g_loss, d_loss = train_step(image_batch, label_batch)
    print(f"Epoch {epoch+1}, Gen Loss: {g_loss.numpy()}, Disc Loss: {d_loss.numpy()}")
    generate_and_plot(generator, epoch+1, label=7)  # Generate hanya angka '7'
